# Prototyping
## Intro
Our project relies on multiple datasets to provide users with flowers based on their desired colors. Research indicates that, surprisingly, there is no unified extensive dataset of flower data and colors, so various public sources are combined.

1. The Flower REflectance Databse, from reflectance.co.uk, is an academic project from Queen Mary, University of London to establish build a database of flower colors. It is the most complete dataset of its kind. The terms of use indicate that it is permissible to use the data for this usecase.

2. The Perenual API from perenual.com has a feature rich and complete dataset with flower info and images. One can make 100 API calls per day for free with a registered account, which is sufficient for this usecase. API queries will be cached locally as the source data is unlikely to change.

### Challenges
* The FReD website is quite old, has no API, and the results page table has malformed HTML. It is necessarry to do some cleanup, parse the HTML, sanitize and enrich the data, then build a relational database from it.
* The FReD database's color field requires some normalization. Some examples:
  * Qualifiers e.g. **dark yellow**, **metallic green**
    * In these cases, the qualifier will be stripped.
  * Compound colors e.g. **white-light purple**, **violet-brown-green**
    * These cases present an opprutinity to enrich and extend the source data by building multiple records for each flower by color.
  * Specific shades e.g. **cream**, **lilac**
    * These will be normalized to CMYK colors.

### Directory layout
```
/prototyping/
├── fred_build_db.sql           # SQL Statements to build the database
├── fred_clean.html             # A manually cleaned HTML document from the FRed site
├── fred_colors.txt             # A textfile of all colors for inspection
├── fred_download.py            # Python script to download the original data from FReD
├── fred_parsed.py              # Python script to process the data step by step with work shown
├── fred_scrape.html            # Original HTML document containing the big table of flowers and colors
├── perenual_get_example.py     # Python example for querying the Perenual API
├── perenual_sample_data.json   # Sample record from Perenual search results
├── prototype.ipynb             # This Jupyter notebook
```

## FReD site to relational DB
### Scrape site

In [2]:
import requests

URL = "http://www.reflectance.co.uk//advanceresults.php?bcolourc=Bee%20Colour&hcolourc=Human%20Colour&maincolourc=Main%20Colour&flowersectc=Flower%20Section&altitudec=&heightc=&tubec=&corollac=&pollinatorc=&familyc=Family&genusc=Genus&speciesc=Species&countryc=Country&townc=&eastc=&southc=&collectorc=&publicationc=&accessionc=&family=*Any%20Family*&genus=*Any%20Genus*&species=*Any%20Species*&country=*Any%20Country*&town=*Any%20Town*&bcolour=*Any%20Colour*&hcolour=*Any%20Colour*&flowersect=*Any%20Section*&pollinator=*Any%20Pollinator*&collector=*Any%20Collector*&maincolour=*Do%20not%20mind*&altitudegreat=-1&altitudeless=2801&heightgreat=-1&heightless=1001&tubegreat=-1&tubeless=1001&corollagreat=-1&corollaless=1001&sort=f.flowerid"

try:
    headers = {'User-Agent': 'student project python script'}
    response = requests.get(URL, headers=headers)
    response.raise_for_status()
except requests.RequestException as e:
    print(f"Error GETting page: {e}")

with open('fred_scrape.html', 'w') as fd:
    fd.write(response.text)

### Clean up data
The website is mostly malformed HTML, and did not play well with lxml or beautifulsoup. The table of results was cleaned using creative text editing and bash stream processing e.g. ```grep "cursor: pointer" | sed s/\<a\ href\"\#\"\>//```

The cleaned HTML table was stored in fred_clean.html.

### Parsing the FReD results
The HTML table can be further processed in Pyhton with XML XPath tree lookups and regular expressions. Here are some relevant snippets from fred_parsed.py

#### Extracting table rows to python array

In [ ]:
# Flowers will be a list of lists to hold the extracted webpage data.
flowers = []

# iterate over rows of the table from the html document (skip the first row)
for row in parsed_doc.getchildren()[1:]:
    # iterate over the columns of each row, building an array of their contents.
    cols = [col.text_content() for col in row.getchildren()]
    # add each array to the flowers array
    flowers.append(cols)

#### Examining the data
There is some weirdness:
* 89 unique colors
* several non-primary colors
* gradient colors separated by hyphen
* 2 entries missing colors entirely

#### Cleanup pass 1: replace "off-white" entries

In [ ]:
for idx, color in enumerate(flowers):
    if 'off-white' in color[2]:
        flowers[idx][2] = 'white'

#### Cleanup pass 2: remove qualifiers ("dark purple", "light-pink")
All qualifiers are in the format "WORD color", if we use the below regex, capture the color, and substitute the backref'd capture, it will clean these up.

In [ ]:
for idx, color in enumerate(flowers):
    flowers[idx][2] = re.sub(r'\b(?:dark|deep|light|pale|white-light|metallic\ light)\s+(\w+)\b', r'\1', color[2], flags=re.IGNORECASE)

#### Cleanup pass 3: Remove compound colors and replace them with multiple entries containg each color
Things like green-yellow-white will be removed and replaced with new entries for each color.

In [ ]:
for idx, color in enumerate(flowers):
    match = re.match(r'^(\w+)(?:-(\w+))+(?:-(\w+))?$', color[2])
    if match:
        for group in match.groups():
            temp_color = color # This is important, you need to copy this before modifying it.
            if group:
                temp_color[2] = group      
                flowers.append(temp_color)
        
        del flowers[idx]      # Now it is safe to remove the compound color entry from flowers.

#### Cleanup final pass: Remove anything that isn't a primary color
There were a few odd colors remaining, but not enough to bother with further.

In [ ]:
acceptable_colors = ['black', 'orange', 'brown', 'purple', 'red', 'blue', 'violet', 'pink', 'white', 'yellow', 'green']
for idx, color in enumerate(flowers):
    if color[2] not in acceptable_colors:
        del flowers[idx]

### Inserting FReD data into relational DB
Python can directly manipulate sqlite dbs, but we are dumping the commands into a sql script for reproducability.